# spark-csv-to-parquet

Converts a CSV file with header to parquet using ApacheSpark

In [ ]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

if [ $version == '36' ]; then
    pip install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1
elif [ $version == '38' ]; then
    pip install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1
else
    echo 'Currently only python 3.6 and 3.8 is supported, in case you need a different version please open an issue at https://github.com/elyra-ai/component-library/issues'
    exit -1
fi

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import os
import sys
import re

In [ ]:
# source path and file name (default: data.csv)
data_csv = os.environ.get('data_csv', 'data.csv')

# destination path and parquet file name (default: data.parquet)
data_parquet = os.environ.get('data_parquet', 'data.parquet')

# url of master (default: local mode)
master = os.environ.get('master', "local[*]")

# temporal data storage for local execution
data_dir = os.environ.get('data_dir', '../../data/')

In [ ]:
# override parameters received from a potential call using %run magic
parameters = list(
  map(
      lambda s: re.sub('$', '"', s),
      map(
          lambda s: s.replace('=', '="'),
          filter(
              lambda s: s.find('=') > -1 and bool(re.match('[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
              sys.argv
          )
      )
  )
)

for parameter in parameters:
    logging.warning('Parameter: '+parameter) 
    exec(parameter)

In [ ]:
skip = False

if os.path.exists(data_dir + data_parquet):
    skip = True

In [ ]:
if not skip:
    sc = SparkContext.getOrCreate(SparkConf().setMaster(master))
    spark = SparkSession.builder.getOrCreate()

In [ ]:
if not skip:
    df = spark.read.option('header', 'true').csv(data_dir + data_csv)

In [ ]:
if not skip:
    df.write.parquet(data_dir + data_parquet)